# Technical Analysis
This is a test on replicating the stage analysis explained in Stan Weinstein's book

## Import all the required Modules

In [1]:
# import the key modules
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from stockbot import preprocessing as prep
from stockbot import data_collection as dc

C:\Users\Rajiv\.conda\envs\stockbot\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Make the style on matplot lib
from matplotlib import style
style.use('ggplot')
%matplotlib inline

### Set logger for the routines

In [3]:
"""
In jupyter notebook simple logging to console and file:
"""
import logging
import sys

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
#         logging.FileHandler(filename='tmp5a.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

# Test
logger = logging.getLogger('LOGGER_NAME')
logger.debug('This message should go to the log file and to the console')
logger.info('So should this')
logger.warning('And this, too')

[{<ipython-input-3-6bc8b2743ab3>:19} INFO - So should this
[{<ipython-input-3-6bc8b2743ab3>:20} WARNING - And this, too


In [4]:
def logged(func):
    def wrapper(*args, **kwargs):
        print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper

## The process to pick stocks
- Get a ticker from the list of tickers
- get the prices for the last 5 years for the ticker
- Calculate the Weighted Moving Average for the stock
- Calculate the slope of MA at every stage
- Collate the volumes
- Calculate support
- Calculate the number of times and the time for which this support has been tested
- Calculate Resistance
- Calculate the number of times and the time for this this resistance held
- Calculate the Trading Range for the stock
- Calculate the market performance and the relative strength
- Spot Continuation points for traders

In [5]:
# Set the ticker and the index ticker
ticker = "XOM"
index_ticker = '^GSPC'

In [6]:
df = prep.prepare_analytical_data(ticker, index_ticker)

[{data_collection.py:45} INFO - end_date: 2020-08-02 00:00:00 and max_index: 2020-07-31 00:00:00
[{data_collection.py:46} INFO - start_date: 2015-08-04 00:00:00 and min_index: 2015-08-04 00:00:00
[{data_collection.py:49} WARNING - Returning an empty dataframe
[{data_collection.py:95} INFO - Dumping data for XOM
[{data_collection.py:45} INFO - end_date: 2020-08-02 00:00:00 and max_index: 2020-07-31 00:00:00
[{data_collection.py:46} INFO - start_date: 2015-08-04 00:00:00 and min_index: 2015-08-04 00:00:00
[{data_collection.py:49} WARNING - Returning an empty dataframe
[{data_collection.py:95} INFO - Dumping data for ^GSPC


In [7]:
print(df.head())

            XOM Adj Close    SMA 10W  Trend SMA 10W  Stage Trend 10W  \
Date                                                                   
2015-08-04      62.495750  62.495750         0.0000              NaN   
2015-08-05      62.495750  62.495750         0.0000              NaN   
2015-08-06      63.240810  62.744104         0.0040              1.0   
2015-08-07      62.220409  62.613180        -0.0021             -1.0   
2015-08-10      63.775311  62.845606         0.0037              1.0   

              SMA 30W  Trend SMA 30W  Stage Trend 30W  XOM Volume  \
Date                                                                
2015-08-04  62.495750         0.0000              NaN  18874400.0   
2015-08-05  62.495750         0.0000              NaN  16544400.0   
2015-08-06  62.744104         0.0040              1.0  13005200.0   
2015-08-07  62.613180        -0.0021             -1.0  12187600.0   
2015-08-10  62.845606         0.0037              1.0  14876000.0   

           

In [8]:
df['Trend SMA 10W'] = df['Trend SMA 10W'].fillna(0)
print(df['Trend SMA 10W'].head())

Date
2015-08-04    0.0000
2015-08-05    0.0000
2015-08-06    0.0040
2015-08-07   -0.0021
2015-08-10    0.0037
Name: Trend SMA 10W, dtype: float64


In [9]:
last_analysis_days=365
df_filtered = df[df.index >= df.index.max() - dt.timedelta(last_analysis_days)]

In [10]:
df_filtered = df

In [11]:
print(df_filtered.head())

            XOM Adj Close    SMA 10W  Trend SMA 10W  Stage Trend 10W  \
Date                                                                   
2015-08-04      62.495750  62.495750         0.0000              NaN   
2015-08-05      62.495750  62.495750         0.0000              NaN   
2015-08-06      63.240810  62.744104         0.0040              1.0   
2015-08-07      62.220409  62.613180        -0.0021             -1.0   
2015-08-10      63.775311  62.845606         0.0037              1.0   

              SMA 30W  Trend SMA 30W  Stage Trend 30W  XOM Volume  \
Date                                                                
2015-08-04  62.495750         0.0000              NaN  18874400.0   
2015-08-05  62.495750         0.0000              NaN  16544400.0   
2015-08-06  62.744104         0.0040              1.0  13005200.0   
2015-08-07  62.613180        -0.0021             -1.0  12187600.0   
2015-08-10  62.845606         0.0037              1.0  14876000.0   

           

In [12]:
start_date, end_date = df.index.min(), df.index.max()

In [13]:
key_ticker_price = '{} Adj Close'.format(ticker)

In [14]:
print(df_filtered[df_filtered['Price History 003M']].tail())

            XOM Adj Close    SMA 10W  Trend SMA 10W  Stage Trend 10W  \
Date                                                                   
2019-03-21      76.321495  69.452304         0.0010              NaN   
2019-04-04      76.564125  70.545471         0.0028              NaN   
2019-04-05      76.974686  70.781412         0.0033              NaN   
2019-04-08      77.450592  70.982886         0.0028              NaN   
2019-04-23      77.805191  72.608906         0.0022              NaN   

              SMA 30W  Trend SMA 30W  Stage Trend 30W  XOM Volume  \
Date                                                                
2019-03-21  72.504603         0.0002              NaN  10117000.0   
2019-04-04  72.616978         0.0002              NaN   9360600.0   
2019-04-05  72.636099         0.0003              NaN  10059100.0   
2019-04-08  72.651543         0.0002              NaN   9108200.0   
2019-04-23  72.759513         0.0003              NaN  10595100.0   

           

### Visualise The Data

In [15]:
%matplotlib widget
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex = True)
fig.set_size_inches(15, 12)
fig.suptitle('{0} - Technical Analysis [from {1} to {2}]'.format(ticker, dt.datetime.strftime(start_date, '%d-%b-%Y'), dt.datetime.strftime(end_date, '%d-%b-%Y')))
num_rows = 9
num_cols = 1
num_rows_ax1 = 5
num_rows_ax2 = 2
num_rows_ax3 = 2
ax1 = plt.subplot2grid((num_rows, num_cols), (0, 0), rowspan=num_rows_ax1, colspan=1)
ax2 = plt.subplot2grid((num_rows, num_cols), (5, 0), rowspan=num_rows_ax2, colspan=1, sharex=ax1)
ax3 = plt.subplot2grid((num_rows, num_cols), (7, 0), rowspan=num_rows_ax3, colspan=1, sharex=ax1)

# Main share price chart
ax1.plot(df_filtered['{} Adj Close'.format(ticker)], label='{} Adj Close'.format(ticker))
ax1.plot(df_filtered['SMA 10W'], label='SMA - 10 week')
ax1.plot(df_filtered['SMA 30W'], label='SMA - 30 week')

ax1.plot(df_filtered[df_filtered['Price History 001M']]['{} Adj Close'.format(ticker)], 'c^', label='Local Maxima 1 M')
ax1.plot(df_filtered[df_filtered['Price History 036M']]['{} Adj Close'.format(ticker)], 'b^', label='Local Maxima 36 M')

ax1.plot(df_filtered[df_filtered['Stage Trend 30W'] == 1]['SMA 30W'], 'g^')
ax1.plot(df_filtered[df_filtered['Stage Trend 30W'] == -1]['SMA 30W'], 'rv')
ax1.plot(df_filtered[df_filtered['Stage Trend 30W'] == 0]['SMA 30W'], 'ys')

ax1.set_title = '{} Adj Close Price History'.format(ticker)
ax1.set_ylabel('Adj. Close Price in USD ($)')
ax1.legend(loc='upper left')

# Volume chart
ax2.bar(df_filtered.index.map(mdates.date2num), df_filtered['{} Volume'.format(ticker)])
ax2.plot(df_filtered['Volume SMA 30D'], label='SMA - 30 day', color='b', alpha=0.3)

ax2.legend(loc='upper left')
ax2.set_ylabel('Volumes (10 mio)')

# Mansfield Relative Strength chart
ax3.plot(df_filtered['RSM'], label='Mansfield RS')
ax3.axhline(y=0, color='g', alpha=0.4)

ax3.legend(loc='upper left')
ax3.set_ylabel('Mansfield RS')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
df_filtered[df_filtered['Stage Trend 30W'] == 1]

,XOM Adj Close,SMA 10W,Trend SMA 10W,Stage Trend 10W,SMA 30W,Trend SMA 30W,Stage Trend 30W,XOM Volume,Volume SMA 30D,^GSPC Adj Close,...,RSM,Price History 1W,Price History 2W,Price History 3W,Price History 001M,Price History 003M,Price History 006M,Price History 012M,Price History 036M,Price History 060M
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-06,63.240810,62.744104,0.0040,1.0,62.744104,0.0040,1.0,13005200.0,1.614133e+07,2083.560059,...,1.208832,True,True,True,True,True,True,True,True,True
2015-08-10,63.775311,62.845606,0.0037,1.0,62.845606,0.0037,1.0,14876000.0,1.509752e+07,2104.179932,...,0.877036,True,True,True,True,True,True,True,True,True
2015-10-02,62.026012,60.749758,0.0005,1.0,60.749758,0.0005,1.0,18811300.0,1.796192e+07,1951.359985,...,3.948507,True,True,True,True,False,False,False,False,False
2015-12-15,65.493889,64.378226,0.0014,NaN,63.628380,0.0003,1.0,24366800.0,1.530246e+07,2043.410034,...,2.051365,True,False,False,False,False,False,False,False,False
2015-12-22,64.026176,64.722638,0.0011,NaN,63.657216,0.0001,1.0,13855800.0,1.593759e+07,2038.969971,...,-0.038297,False,False,False,False,False,False,False,False,False
2016-01-05,64.413727,65.295813,0.0011,NaN,63.745372,0.0001,1.0,11993500.0,1.557990e+07,2016.709961,...,1.623236,False,False,False,False,False,False,False,False,False
2016-01-14,65.238266,65.491856,0.0005,NaN,63.679436,0.0002,1.0,33806700.0,1.825821e+07,1921.839966,...,7.794528,True,False,False,False,False,False,False,False,False
2016-01-29,64.191086,65.086687,-0.0004,NaN,63.588638,0.0001,1.0,25161900.0,1.899637e+07,1940.239990,...,4.625750,True,False,False,False,False,False,False,False,False
2016-02-03,64.710556,64.939895,-0.0007,NaN,63.575665,0.0001,1.0,25873700.0,1.901635e+07,1912.530029,...,6.901436,True,False,False,False,False,False,False,False,False


In [17]:
df_sandp = dc.get_ticker_data_from_yahoo(ticker)

[{data_collection.py:45} INFO - end_date: 2020-08-02 00:00:00 and max_index: 2020-07-31 00:00:00
[{data_collection.py:46} INFO - start_date: 2015-08-04 00:00:00 and min_index: 2015-08-04 00:00:00
[{data_collection.py:49} WARNING - Returning an empty dataframe
[{data_collection.py:95} INFO - Dumping data for XOM


In [18]:
df_sandp[-1000:].Close

Date
2016-08-11    86.720001
2016-08-12    87.849998
2016-08-15    87.809998
2016-08-16    87.919998
2016-08-17    88.110001
                ...    
2020-07-27    44.070000
2020-07-28    43.549999
2020-07-29    44.029999
2020-07-30    41.869999
2020-07-31    42.080002
Name: Close, Length: 1000, dtype: float64

### trendln

Check out the support and resistance file at trendln: https://github.com/GregoryMorse/trendln

In [19]:
df['SMA 30W']

Date
2015-08-04    62.495750
2015-08-05    62.495750
2015-08-06    62.744104
2015-08-07    62.613180
2015-08-10    62.845606
                ...    
2020-07-27    54.199853
2020-07-28    54.084297
2020-07-29    53.968705
2020-07-30    53.846787
2020-07-31    53.733422
Name: SMA 30W, Length: 1258, dtype: float64

In [20]:
df['SMA 30W'].rolling(window=2).aggregate(lambda x: (x[1] - x[0]) / x[0])

Date
2015-08-04         NaN
2015-08-05    0.000000
2015-08-06    0.003974
2015-08-07   -0.002087
2015-08-10    0.003712
                ...   
2020-07-27   -0.002114
2020-07-28   -0.002132
2020-07-29   -0.002137
2020-07-30   -0.002259
2020-07-31   -0.002105
Name: SMA 30W, Length: 1258, dtype: float64

In [21]:
df[df['Trend SMA 30W'].rolling(window=2).aggregate(lambda x: ((x[0] < 0 and x[1] > 0) or (x[0] > 0 and x[1] < 0))).astype(bool)].index

DatetimeIndex(['2015-08-04', '2015-08-07', '2015-08-10', '2015-08-19',
               '2015-10-02', '2015-12-07', '2015-12-15', '2016-01-14',
               '2016-02-01', '2016-02-03', '2018-04-27', '2019-01-22'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [22]:
df['Trend SMA 30W Rounded'] = round(df['Trend SMA 30W'], 4) 

In [23]:
df[df['Trend SMA 30W Rounded'].rolling(window=2).aggregate(lambda x: ((x[0] < 0 and x[1] > 0) or (x[0] > 0 and x[1] < 0))).astype(bool)].index

DatetimeIndex(['2015-08-04', '2015-08-07', '2015-08-10', '2015-08-19',
               '2015-10-02', '2015-12-07', '2015-12-15', '2016-01-14',
               '2016-02-01', '2016-02-03', '2018-04-27', '2019-01-22'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [24]:
df[df['Trend SMA 30W Rounded'].rolling(window=2).aggregate(lambda x: (x[0] == 0 or x[1] == 0)).astype(bool)].index

DatetimeIndex(['2015-08-04', '2015-08-05', '2015-08-06', '2015-09-30',
               '2015-10-01', '2015-12-18', '2015-12-21', '2015-12-22',
               '2016-01-04', '2016-01-05',
               ...
               '2019-11-01', '2019-11-13', '2019-11-14', '2019-11-15',
               '2019-11-18', '2019-11-20', '2019-11-21', '2019-11-26',
               '2019-11-27', '2019-11-29'],
              dtype='datetime64[ns]', name='Date', length=133, freq=None)

In [25]:
len(df[df['Trend SMA 30W Rounded'] == 0].index.tolist())

87

In [26]:
print(df[df.index > pd.to_datetime('2018-03-01')][:10])

            XOM Adj Close    SMA 10W  Trend SMA 10W  Stage Trend 10W  \
Date                                                                   
2018-03-02      67.634674  73.310517        -0.0007              NaN   
2018-03-05      68.279236  73.266207        -0.0006              NaN   
2018-03-06      68.198662  73.216696        -0.0007              NaN   
2018-03-07      66.479813  73.139718        -0.0011              NaN   
2018-03-08      66.354492  73.056898        -0.0011              NaN   
2018-03-09      66.748390  72.983630        -0.0010              NaN   
2018-03-12      67.357132  72.911969        -0.0010              NaN   
2018-03-13      66.721535  72.823633        -0.0012              NaN   
2018-03-14      65.888962  72.710489        -0.0016              NaN   
2018-03-15      66.623055  72.605681        -0.0014              NaN   

              SMA 30W  Trend SMA 30W  Stage Trend 30W  XOM Volume  \
Date                                                              

In [27]:
def mark_trend_turn(x):
    if (x[0] <= 0 and x[1] > 0):
        return 1
    elif (x[0] != 0 and x[1] == 0):
        return 0
    elif (x[0] >= 0 and x[1] < 0):
        return -1
    else:
        return np.NaN

In [28]:
df['Trend SMA 30W'].rolling(window=2).apply(mark_trend_turn).value_counts()

 0.0    45
 1.0    29
-1.0    28
Name: Trend SMA 30W, dtype: int64